<img src="../images/logos/xradar_logo.svg" width=250 alt="xradar"></img>

# Xradar Basics


---

## Overview

1. Xradar general overview
1. Radar data IO
2. Radar data georeferencing
3. Data visualization
   

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray.html) | Necessary |  Basic features |
| [Radar Cookbook](https://projectpythia.org/radar-cookbook/README.html) | Necessary |  Radar basics |
| [Matplotlib](https://foundations.projectpythia.org/core/matplotlib.html) | Necessary |  Plotting basic features |
- **Time to learn**: 30 minutes
---

## Imports

In [ ]:
import xradar as xd
import pyart
import wradlib as wrl

import fsspec
import numpy as np
from xarray.backends.api import open_datatree

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## Xradar

`Xradar` is a `Python` package developed to streamline the **reading** and **writing** of radar data across various formats, with exports aligned to standards like ODIM_H5 and CfRadial. Born from the **Open Radar Science Community's** collaboration at **ERAD2022**, `Xradar` uses an `xarray-based` data model, compatible with the upcoming **CfRadial2.1/FM301** standard. This ensures seamless integration with other xarray-based software and existing open-source radar processing tools.

### Xradar data model
Xradar leverages the upcoming FM301 standard, a subset of CfRadial2.0, using Xarray to efficiently manage radar data.

#### DataTree Structure (CfRadial2.1/FM301 standard)
Xradar employs `xarray.DataTree` objects to organize radar sweeps within a single structure, where each sweep is an `xarray.Dataset` containing relevant metadata and variables.

<img src="../images/CfRadial2.1.svg" width=500 alt="xradar"></img>

## Xradar importers

Xradar supports several importers to handle different radar data formats. These importers typically include:

- **ODIM_H5**: For reading radar data in the ODIM_H5 format, which is widely used in Europe and supported by the EUMETNET OPERA program.

- **CfRadial**: For importing data in the CfRadial format, which is commonly used in the atmospheric sciences community.

- **SIGMET/IRIS**: For ingesting radar data from SIGMET/IRIS formats, which are used by various weather radar systems globally.

- **GAMIC**: For reading data from GAMIC radars, which use their proprietary format.

For more importers check [here](https://docs.openradarscience.org/projects/xradar/en/stable/importers.html)

Let's find some Italian radar data located at the Erad 2024 Bucket

In [ ]:
# Set the URL and path for the cloud
URL = 'https://js2.jetstream-cloud.org:8001/'
path = f'pythia/radar/erad2024'

fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))

fs.glob(f"{path}/*")

In [ ]:
files = fs.glob("pythia/radar/erad2024/20240522_MeteoSwiss_ARPA_Lombardia/Data/Cband/*.nc")
files[:3]

In [ ]:
radar_files = [f"s3://{i}" for i in files]
radar_files[:3]

In [ ]:
local_files = [
    fsspec.open_local(
        f"simplecache::{URL}{i}", s3={"anon": True}, filecache={"cache_storage": "."}
    )
    for i in files[:5]
]

We can open one of this `nc` files using `xradar.io.open_cfradial1_datree` method

In [ ]:
dt = xd.io.open_cfradial1_datatree(local_files[0])
display(dt)

In this `Xarray.Datatree` object, the first two nodes contains radar metadata and the others contains `Xarray.Datasets` for each sweeps. 

In [ ]:
for sweep in dt.children:
    try:
        print(f"{sweep} - elevation {np.round(dt[sweep]['sweep_fixed_angle'].values[...], 1)}")
    except KeyError:
        print (sweep)

Let's explore the `1.0` degrees elevation ('sweep_2')

In [ ]:
ds_sw2 = dt["sweep_2"].ds
display(ds_sw2)

## Xradar visualization

We can make a plot using [`xarray.plot`](https://docs.xarray.dev/en/latest/user-guide/plotting.html) functionality.

In [ ]:
ds_sw2.reflectivity.plot(
    cmap="ChaseSpectral",
    vmax=60,
    vmin=-10, 
)

The radar data in the `Xarray.Dataset` object includes range and azimuth as coordinates. To create a radial plot, apply the [`xradar.georeference`](https://docs.openradarscience.org/projects/xradar/en/stable/georeference.html) method. This method will generate x, y, and z coordinates for the plot.

In [ ]:
ds_sw2 = ds_sw2.xradar.georeference()
display(ds_sw2)

We can now create a radial plot passing `x` and `y` coordinates

In [ ]:
ds_sw2.reflectivity.plot(
    x="x", 
    y= "y",
    cmap="ChaseSpectral",
    vmax=60,
    vmin=-10, 
)

## Data slicing

We can use the power of `Xarray` to acces data within the first 50 kilometers by [slicing](https://foundations.projectpythia.org/core/xarray/xarray-intro.html#slicing-along-coordinates) along coordinates.

In [ ]:
ds_sw2.sel(range=slice(0, 5e4)).reflectivity.plot(
    x="x", 
    y= "y",
    cmap="ChaseSpectral",
    vmax=60,
    vmin=-10, 
)

Let's suposse we want to subset between 90 and 180 degrees angle in azumith

In [ ]:
ds_sw2.sel(azimuth=slice(90, 180), range=slice(0, 5e4)).reflectivity.plot(
    x="x", 
    y= "y",
    cmap="ChaseSpectral",
    vmax=60,
    vmin=-10, 
)

Perhaps, we just what to see radar reflectivity along the 100 degrees angle in azimuth

In [ ]:
ds_sw2.sel(azimuth=100, method="nearest").reflectivity.plot()

## Xradar integration

### Py-Art

`Xradar` datatree objects can be ported to `Py-ART` radar objects using the pyart.xradar.Xradar method.

In [ ]:
radar = pyart.xradar.Xradar(dt)
fig = plt.figure(figsize=[10, 8])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map('reflectivity', sweep=0)

In [ ]:
del display

### Wradlib

`Wradlib` functionality can also be applied to `Xarray.Datatree` objects. For example, the [`wradlib.georef`](https://docs.wradlib.org/en/latest/georef.html) module can be used to enrich data by adding geographical coordinates.

In [ ]:
for key in list(dt.children):
    if "sweep" in key:
        dt[key].ds = dt[key].ds.wrl.georef.georeference(
            crs=wrl.georef.get_default_projection()
        )

In [ ]:
dt["sweep_0"].ds.reflectivity.sel(range=slice(0, 5e4)).plot(
    x="x", 
    y= "y",
    cmap="ChaseSpectral",
    vmax=60,
    vmin=-10, 
)

## Xradar exporters

In xradar, exporters convert radar data into various formats for analysis or integration. Exporting is supported for recognized standards, including:

- CfRadial1
- CfRadial2
- ODIM
- Zarr

Although [`Zarr`](https://zarr.readthedocs.io/en/stable/getting_started.html) is not a traditional standard format, it provides an analysis-ready, cloud-optimized format that enhances data accessibility and performance. 

In [ ]:
dt.to_zarr("tree.zarr", consolidated=True, mode="w")

In [ ]:
dt_back = open_datatree("tree.zarr", engine="zarr",chunks={})

In [ ]:
display(dt_back)

---

## Summary

`Xradar` is a Python library designed for working with radar data. It extends xarray to include radar-specific functionality, such as purpose-based accessors and georeferencing methods. It supports exporting data in various formats, including CfRadial1, CfRadial2, ODIM, and Zarr. xradar facilitates the analysis, visualization, and integration of radar data with other tools and systems.

## Resources and references
 - [Xradar](https://docs.openradarscience.org/projects/xradar/en/stable/index.html)
 - [Radar cookbook](https://github.com/ProjectPythia/radar-cookbook)
 - [Py-Art landing page](https://arm-doe.github.io/pyart/)
 - [Wradlib landing page](https://docs.wradlib.org/en/latest/index.html)
